In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


### 1. Load data and split event / non-event in order to handle imbalanced data

#### 1.1 Load data 

In [2]:
df_clean = pd.read_csv('clean_training_data.csv',
                 converters={'id': str, 
                             'member_id': str})
df_clean.head()

,id,member_id,loan_amnt,emp_length,home_ownership,annual_inc,desc,purpose,dti,inq_last_6mths,...,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,revol_util_nb_fixed,loan_amnt_log,annual_inc_log,total_bc_limit_log,tot_hi_cred_lim_log,tot_cur_bal_log
0,10000001,11983056,7550,3 years,RENT,28000.0,NaN,debt_consolidation,8.40,0.0,...,4000.0,41.792473,3828.953801,5759.0,9600.0,8.929303,10.239960,8.294050,8.250347,8.658519
1,10000002,12002921,27050,10+ years,OWN,55000.0,Borrower added on 12/31/13 > Combining high ...,debt_consolidation,22.87,0.0,...,35700.0,41.792473,34359.940730,114834.0,5390.0,10.205442,10.915088,10.482906,10.444647,11.651243
2,10000003,11983096,12000,4 years,RENT,60000.0,Borrower added on 12/31/13 > I would like to...,debt_consolidation,4.62,1.0,...,18100.0,41.792473,16416.617760,7137.0,1590.0,9.392662,11.002100,9.803667,9.706049,8.873048
3,10000004,12003142,28000,5 years,MORTGAGE,325000.0,NaN,debt_consolidation,18.55,1.0,...,42200.0,41.792473,38014.149760,799592.0,6710.0,10.239960,12.691580,10.650176,10.545714,13.591857
4,10000005,11993233,12000,10+ years,RENT,40000.0,NaN,debt_consolidation,16.94,0.0,...,7000.0,53.000000,6471.462236,13605.0,7960.0,9.392662,10.596635,8.853665,8.775157,9.518193


#### 1.2 Split Event / Non-Event 

In [3]:
df_event = df_clean.loc[df_clean['bad_flag'] == 1].copy()
df_nonevent = df_clean.loc[df_clean['bad_flag'] != 1].copy()
len(df_event), len(df_nonevent)

(13128, 174995)

#### 1.3 Bootstrap rare samples to make a balanced data set

In [4]:
bootstrap_samples = []
for _ in range(27):
    sample = df_event.sample(frac=0.5, replace=True, random_state=44)
    bootstrap_samples.append(sample)

df_event_up = pd.concat(bootstrap_samples)

In [5]:
df = pd.concat([df_nonevent, df_event_up], ignore_index=True)
df.bad_flag.value_counts()

bad_flag
1.0    177228
0.0    174995
Name: count, dtype: int64

### 2. Engineer New Features
Findings / Summary:
* This is the fun part. It takes time and innovation to engineer useful features. For illustration purpose, I did 3 examples as follows.
    * Encoding categorical variables
    * Creating interactive variables
    * Binning

#### 2.1 Encode categorical variables

In [6]:
# Encode categorical variables
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

categorical_vars = ['emp_length', 'home_ownership', 'purpose']
# Encode the categorical variable
for v in categorical_vars:
    print(f"--- Encoding {v} ---")
    df[v + '_encoded'] = label_encoder.fit_transform(df[v])
    print(df[v + '_encoded'].value_counts())

--- Encoding emp_length ---
emp_length_encoded
1     109674
2      30572
5      27138
10     27021
3      25846
4      22724
6      22476
0      21365
7      20027
8      16443
11     15644
9      13293
Name: count, dtype: int64
--- Encoding home_ownership ---
home_ownership_encoded
0    166831
4    156289
3     28889
1       119
2        95
Name: count, dtype: int64
--- Encoding purpose ---
purpose_encoded
2     210788
1      70938
8      20868
3      17251
10      8432
5       6839
0       3658
6       3444
12      2884
4       2669
7       2387
11      1773
9        292
Name: count, dtype: int64


#### 2.2  Create interactive variable

In [7]:
# for example: 
df['bc_util_fixed_X_revol_util_nb_fixed'] = df['bc_util_fixed'] * df['revol_util_nb_fixed']

#### 2.3 Binning

In [8]:
# create bins for annual_inc based off 3 quartiles
q1 = df['annual_inc'].quantile(0.25)
q2 = df['annual_inc'].quantile(0.5)
q3 = df['annual_inc'].quantile(0.75)

c1 = df['annual_inc'] < q1
c2 = (df['annual_inc'] >= q1) & (df['annual_inc'] < q2)
c3 = (df['annual_inc'] >= q2) & (df['annual_inc'] < q3)
c4 = df['annual_inc'] >= q3

df['annual_inc_binned'] = np.select([c1, c2, c3, c4], [1, 2, 3, 4], default=0.0)


In [9]:
# Let's look at the bad rate of each bin -> looks like lower income borrowers have higher probability to default at their loans.
df.groupby(['annual_inc_binned'])['bad_flag'].sum() / df.groupby(['annual_inc_binned'])['bad_flag'].count()

annual_inc_binned
1.0    0.579547
2.0    0.535462
3.0    0.471842
4.0    0.427088
Name: bad_flag, dtype: float64

In [10]:
# output dataset
df.to_csv('df_balanced_with_new_features.csv', index=False)